In [1]:
%matplotlib inline
import pandas as pd
import pylab as p
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import median_absolute_error, mean_squared_error, explained_variance_score, mean_absolute_error, r2_score
from sklearn.cross_validation import ShuffleSplit, train_test_split
import scipy
import random
import time
from multiprocessing import Pool
import matplotlib
matplotlib.style.use('ggplot')

In [2]:
game_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/GAMES.csv', index_col=[0], na_values="\N" )
game_df.drop(['V', 'H'], axis=1, inplace=True)
game_df = pd.get_dummies(game_df, columns=['DAY', 'STAD', 'WDIR', 'COND', 'SURF'])

In [3]:
player_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PLAYERS.csv', index_col=[0])
player_df.drop(['FNAME', 'LNAME', 'PNAME'], axis=1, inplace=True)
player_df = pd.get_dummies(player_df, columns=['POS1', 'POS2', 'COL'])

In [4]:
plays_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/CORE.csv', index_col=[1])
plays_df = pd.get_dummies(plays_df, columns=['OFF', 'DEF', 'TYPE'])

In [5]:
pass_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PASS.csv', index_col=[0])
pass_df['PID'] = pass_df.index
pass_df.set_index('PSR', inplace=True)
pass_df = pass_df.join(player_df, rsuffix="_pass_psr")
pass_df['PSR'] = pass_df.index
pass_df.set_index('TRG', inplace=True)
pass_df = pass_df.join(player_df, rsuffix="_pass_trg")
pass_df['TRG'] = pass_df.index
pass_df.drop(['PSR', 'TRG'], axis=1, inplace=True)
pass_df = pd.get_dummies(pass_df, columns=['LOC'])

In [6]:
rush_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/RUSH.csv', index_col=[0])
rush_df['PID'] = rush_df.index
rush_df.set_index('BC', inplace=True)
rush_df = rush_df.join(player_df, rsuffix="_rush_bc")
rush_df['BC'] = rush_df.index
rush_df.drop(['BC'], axis=1, inplace=True)
rush_df = pd.get_dummies(rush_df, columns=['DIR'])

In [7]:
# player does not need to be joined either a rusher, passer or ball carrier
conv_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/CONVS.csv', index_col=[0], true_values=['Y'], false_values=['N'])
conv_df.drop(['BC', 'PSR', 'TRG'], axis=1, inplace=True)
conv_df = pd.get_dummies(conv_df, columns=['TYPE'])

In [8]:
kickoff_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/KICKOFFS.csv', index_col=[0], true_values=['Y'], false_values=['N'])
kickoff_df['PID'] = kickoff_df.index
kickoff_df.set_index('KR', inplace=True)
kickoff_df = kickoff_df.join(player_df, rsuffix="_kickoff_returner")
kickoff_df['KR'] = kickoff_df.index
kickoff_df.set_index('KICKER', inplace=True)
kickoff_df = kickoff_df.join(player_df, rsuffix="_kickoff_kicker")
kickoff_df['KICKER'] = kickoff_df.index
kickoff_df.drop(['KICKER', 'KR'], axis=1, inplace=True)

In [9]:
punt_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PUNTS.csv', index_col=[0], true_values=['Y'], false_values=['N'])
punt_df['PID'] = punt_df.index
punt_df.set_index('PR', inplace=True)
punt_df = punt_df.join(player_df, rsuffix="_punt_returner")
punt_df['PR'] = punt_df.index
punt_df.set_index('PUNTER', inplace=True)
punt_df = punt_df.join(player_df, rsuffix="_punt_kicker")
punt_df['PUNTER'] = punt_df.index
punt_df.drop(['PR', 'PUNTER'], axis=1, inplace=True)

In [10]:
tackle_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/TACKLES.csv', index_col=[0])
tackle_df['PID'] = tackle_df.index
tackle_df.set_index('TCK', inplace=True)
tackle_df = tackle_df.join(player_df, rsuffix="_tackler")
tackle_df['TCK'] = tackle_df.index
tackle_df.drop('TCK', axis=1, inplace=True)

In [11]:
intercept_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/INTS.csv', index_col=[0])
intercept_df['PID'] = intercept_df.index
intercept_df.set_index('INT', inplace=True)
intercept_df = intercept_df.join(player_df, rsuffix="_int")
intercept_df['INT'] = intercept_df.index
intercept_df.drop(['INT'], axis=1, inplace=True)

In [12]:
fumble_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/FUMBLES.csv', index_col=[0])
fumble_df['PID'] = fumble_df.index
fumble_df.set_index('FUM', inplace=True)
fumble_df = fumble_df.join(player_df, rsuffix="_fum")
fumble_df['FUM'] = fumble_df.index
fumble_df.set_index('FRCV', inplace=True)
fumble_df = fumble_df.join(player_df, rsuffix="_fum")
fumble_df['FRCV'] = fumble_df.index
fumble_df.drop(['FUM', 'FRCV'], axis=1, inplace=True)

In [13]:
penalty_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/PENALTIES.csv', index_col=[0])
penalty_df['PID'] = penalty_df.index
penalty_df.set_index('PEN', inplace=True)
penalty_df = penalty_df.join(player_df, rsuffix="_pen")
penalty_df['PEN'] = penalty_df.index
penalty_df.drop(['PEN'], axis=1, inplace=True)
penalty_df = pd.get_dummies(penalty_df, columns=['DESC', 'ACT', 'PTM'])

In [14]:
fg_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/FGXP.csv', index_col=[0], true_values=['Y'], false_values=['N'])
fg_df['PID'] = fg_df.index
fg_df.set_index('FKICKER', inplace=True)
fg_df = fg_df.join(player_df, rsuffix="_FKICKER")
fg_df['FKICKER'] = fg_df.index
fg_df.drop('FKICKER', axis=1, inplace=True)
fg_df = pd.get_dummies(fg_df, columns=['FGXP'])

In [15]:
block_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/BLOCKS.csv', index_col=[0])
block_df['PID'] = block_df.index
block_df.set_index('BLK', inplace=True)
block_df = block_df.join(player_df, rsuffix="_BLK")
block_df['BLK'] = block_df.index
block_df.set_index('BRCV', inplace=True)
block_df = block_df.join(player_df, rsuffix="_BRCV")
block_df['BRCV'] = block_df.index
block_df.drop(['BLK', 'BRCV'], axis=1, inplace=True)

In [16]:
safety_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/SAFETIES.csv', index_col=[0])
safety_df['PID'] = safety_df.index
safety_df.set_index('SAF', inplace=True)
safety_df = safety_df.join(player_df, rsuffix="_SAF")
safety_df['SAF'] = safety_df.index
safety_df.drop(['SAF'], axis=1, inplace=True)

In [17]:
sack_df = pd.read_csv('/Users/jostheim/workspace/kaggle/data/nfl/NFLData_2000-2012/SACKS.csv', index_col=[0])
sack_df['PID'] = sack_df.index
sack_df.set_index('SK', inplace=True)
sack_df = sack_df.join(player_df, rsuffix="_SK")
sack_df['SK'] = sack_df.index
sack_df.set_index('QB', inplace=True)
sack_df = sack_df.join(player_df, rsuffix="_QB")
sack_df['QB'] = sack_df.index
sack_df.drop(['SK', 'QB'], axis=1, inplace=True)

In [ ]:
plays_df = plays_df.join(conv_df, rsuffix="_conv")
plays_df = plays_df.join(kickoff_df, rsuffix="_kickoff")
plays_df = plays_df.join(punt_df, rsuffix="_punt")
plays_df = plays_df.join(tackle_df, rsuffix="_tackle")
plays_df = plays_df.join(intercept_df, rsuffix="_intercept")
plays_df = plays_df.join(fumble_df, rsuffix="_fumble")
plays_df = plays_df.join(pass_df, rsuffix="_pass")
plays_df = plays_df.join(rush_df, rsuffix="_rush")
plays_df = plays_df.join(penalty_df, rsuffix="_penalty")
plays_df = plays_df.join(fg_df, rsuffix="_fg")
plays_df = plays_df.join(block_df, rsuffix="_block")
plays_df['PID'] = plays_df.index
plays_df.set_index("GID", inplace=True)
plays_df = plays_df.join(game_df, rsuffix="_game")
plays_df['GID'] = plays_df.index
plays_df.drop(['PID_conv', 'PID_kickoff', 'PID_punt', 'PID_tackle', 'PID_intercept', 'PID_fumble', 'PID_pass', 'PID_rush', 'PID_penalty', 'PID_fg', 'PID_block'])

In [ ]:
for i, col in enumerate(plays_df.columns):
    print col, plays_df.dtypes[i]

In [ ]:
print plays_df